## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP1 - Verificación de la calidad de los datos (versión completa)

---

En esta libreta se realiza una verificación de calidad de datos con base en los planteado en el Anexo SP4. 

---

In [1]:
#Se importan las librerias a utilizar

import pandas as pd

----

##### Lectura de los datasets

In [2]:
df_creditos = pd.read_csv("../../data/processed/datos_creditos.mc.csv", sep=";")
display(df_creditos.head(1))

df_tarjetas = pd.read_csv("../../data/processed/datos_tarjetas.mc.csv", sep=";")
display(df_tarjetas.head(1))

id_cliente  edad  importe_solicitado  duracion_credito  \
0  713061558.0    22               35000                 3   

   antiguedad_empleado situacion_vivienda  ingresos objetivo_credito  \
0                123.0           ALQUILER     59000         PERSONAL   

   pct_ingreso  tasa_interes  estado_credito falta_pago  
0         0.59         16.02               1          Y

id_cliente  antiguedad_cliente estado_civil estado_cliente  \
0  713061558.0                36.0       CASADO         ACTIVO   

   gastos_ult_12m genero  limite_credito_tc         nivel_educativo  \
0          1088.0      M             4010.0  UNIVERSITARIO_COMPLETO   

  nivel_tarjeta  operaciones_ult_12m  personas_a_cargo  
0          Blue                 24.0               2.0

---
#### Verificación de calidad de datos

**Análisis a realizar**

1. Evaluación de valores nulos (filas y columnas)
2. Evaluación de formato válido
3. Valores ajustados en rangos (ver anexos)
4. Claves únicas
5. Integridad referencial
6. Cumplimiento de reglas en valores

In [3]:
# Establecimiento de los umbrales de aceptación

FORMATEO_VALORES = 0.1
RANGOS_VALORES = 0.0
INTEGRIDAD_REF = 0.1
REGLAS_VALORES = 0.1

# Valores globales

cantidad_filas_creditos = df_creditos.shape[0]
cantidad_filas_tarjetas = df_tarjetas.shape[0]

## Dimensión: completitud

### (1a) Filas

In [4]:
# Se obtienen las cantidades de valores nulos por columna

nulos_x_columna_c = df_creditos.isna().sum()
nulos_x_columna_t = df_tarjetas.isna().sum()

print(f"Cantidad de filas que tienen valores nulos por atributo:\n{nulos_x_columna_c}\n")
print(f"Cantidad de filas que tienen valores nulos por atributo:\n{nulos_x_columna_t}")

Cantidad de filas que tienen valores nulos por atributo:
id_cliente               0
edad                     0
importe_solicitado       0
duracion_credito         0
antiguedad_empleado    337
situacion_vivienda       0
ingresos                 0
objetivo_credito         0
pct_ingreso              0
tasa_interes           912
estado_credito           0
falta_pago               0
dtype: int64

Cantidad de filas que tienen valores nulos por atributo:
id_cliente             0
antiguedad_cliente     0
estado_civil           0
estado_cliente         0
gastos_ult_12m         0
genero                 0
limite_credito_tc      0
nivel_educativo        0
nivel_tarjeta          0
operaciones_ult_12m    0
personas_a_cargo       0
dtype: int64


In [5]:
# De cualquier manera se establece el cálculo a realizar

cantidad_columnas = len(df_creditos.axes[1])

df_creditos['completitud_fila'] = (df_creditos.isnull().sum(axis=1) / cantidad_columnas)

problemas = df_creditos[df_creditos['completitud_fila'] >= 0.2]

completitud_f = problemas.shape[0]

print(f"Filas que incumplen el umbral de nulos en columnas [completitud_f] - créditos - :")
print(f"{completitud_f} ({round((completitud_f  / cantidad_filas_creditos) * 100, 2)})%")

Filas que incumplen el umbral de nulos en columnas [completitud_f] - créditos - :
0 (0.0)%


In [6]:
# De cualquier manera se establece el cálculo a realizar

cantidad_columnas = len(df_tarjetas.axes[1])

df_tarjetas['completitud_fila'] = (df_tarjetas.isnull().sum(axis=1) / cantidad_columnas)

problemas = df_tarjetas[df_tarjetas['completitud_fila'] >= 0.2]

completitud_f = problemas.shape[0]

print(f"Filas que incumplen el umbral de nulos en columnas [completitud_f] - tarjetas - :")
print(f"{completitud_f} ({round((completitud_f  / cantidad_filas_creditos) * 100, 2)})%")

Filas que incumplen el umbral de nulos en columnas [completitud_f] - tarjetas - :
0 (0.0)%


### (1b) Dataset

In [7]:
completitud_dc = df_creditos.isnull().any(axis=1).sum()

print(f"Filas que presentan nulos en el dataset [completitud_d] - creditos - :")
print(f"{completitud_dc} ({round((completitud_dc  / cantidad_filas_creditos) * 100, 2)})%\n")

completitud_dt = df_tarjetas.isnull().any(axis=1).sum()

print(f"Filas que presentan nulos en el dataset [completitud_d] - tarjetas - :")
print(f"{completitud_dt} ({round((completitud_dt  / cantidad_filas_tarjetas) * 100, 2)})%")

Filas que presentan nulos en el dataset [completitud_d] - creditos - :
1225 (12.1)%

Filas que presentan nulos en el dataset [completitud_d] - tarjetas - :
0 (0.0)%


----

## Dimensión: exactitud

### (2) Formato válido

In [8]:
# No se encuentran atributos con formato específico

----

## Dimensión: exactitud

### (3) Valores ajustados

Atributo: **edad**

In [9]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: edad

valores = df_creditos['edad'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['edad']) - df_creditos['edad'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['edad'] > 90]

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
edad
22     2228
23     2190
24     1926
25     1646
26     1328
21      799
20        6
144       2
123       2
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


id_cliente  edad  importe_solicitado  duracion_credito  \
81   715623483.0   144                4800                 3   
183  815500983.0   144                6000                 2   
575  717956283.0   123               20400                 3   
747  712881483.0   123               20000                 4   

     antiguedad_empleado situacion_vivienda  ingresos objetivo_credito  \
81                   4.0           ALQUILER    250000      INVERSIONES   
183                  4.0           HIPOTECA    200000        EDUCACIÓN   
575                  2.0           ALQUILER     80004        EDUCACIÓN   
747                  7.0           ALQUILER     78000      INVERSIONES   

     pct_ingreso  tasa_interes  estado_credito falta_pago  completitud_fila  
81          0.02         13.57               0          N          0.000000  
183         0.03         11.86               0          N          0.000000  
575         0.25         10.25               0          N          0.000000  
747         0.26           NaN               0          N          0.083333

Cantidad detectada: 4


In [12]:
def calcular_rangos_valores_edad():
    edad_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo edad: {edad_valores_fuera_rango}")
    
    indicador = (edad_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo edad): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_edad()

Cantidad de filas con valores fuera de rango en atributo edad: 0
Porcentaje de filas con errores de rango de valores (atributo edad): 0.0 %
Evaluación: ok


In [10]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: importe

valores = df_creditos['importe_solicitado'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['importe_solicitado']) - df_creditos['importe_solicitado'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['importe_solicitado'] < 0]

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
importe_solicitado
5000     800
6000     715
8000     556
4000     446
7000     436
        ... 
750        1
700        1
19800      1
4150       1
9425       1
Name: count, Length: 509, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


Empty DataFrame
Columns: [id_cliente, edad, importe_solicitado, duracion_credito, antiguedad_empleado, situacion_vivienda, ingresos, objetivo_credito, pct_ingreso, tasa_interes, estado_credito, falta_pago, completitud_fila]
Index: []

Cantidad detectada: 0


In [11]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: duracion_credito

valores = df_creditos['duracion_credito'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['duracion_credito']) - df_creditos['duracion_credito'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['duracion_credito'] < 0]

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
duracion_credito
2    3404
3    3364
4    3359
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


Empty DataFrame
Columns: [id_cliente, edad, importe_solicitado, duracion_credito, antiguedad_empleado, situacion_vivienda, ingresos, objetivo_credito, pct_ingreso, tasa_interes, estado_credito, falta_pago, completitud_fila]
Index: []

Cantidad detectada: 0


In [13]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: antiguedad_empleado

valores = df_creditos['antiguedad_empleado'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['antiguedad_empleado']) - df_creditos['antiguedad_empleado'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['antiguedad_empleado'] > 50]

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
antiguedad_empleado
0.0      1414
2.0      1281
3.0      1128
5.0      1050
6.0      1027
1.0       993
4.0       846
7.0       784
8.0       587
9.0       414
10.0      237
11.0       27
123.0       2
Name: count, dtype: int64

Cantidad de nulos en el atributo: 337

Se visualizan las filas con errores de rango:


id_cliente  edad  importe_solicitado  duracion_credito  \
0    713061558.0    22               35000                 3   
210  715999608.0    21               20000                 4   

     antiguedad_empleado situacion_vivienda  ingresos objetivo_credito  \
0                  123.0           ALQUILER     59000         PERSONAL   
210                123.0           HIPOTECA    192000      INVERSIONES   

     pct_ingreso  tasa_interes  estado_credito falta_pago  completitud_fila  
0           0.59         16.02               1          Y               0.0  
210         0.10          6.54               0          N               0.0

Cantidad detectada: 2


In [14]:
def calcular_rangos_valores_antiguedad_empleado():
    antiguedad_empleado_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo antigueda empleado: {antiguedad_empleado_valores_fuera_rango}")
    
    indicador = (antiguedad_empleado_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo edad): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_antiguedad_empleado()

Cantidad de filas con valores fuera de rango en atributo antigueda empleado: 339
Porcentaje de filas con errores de rango de valores (atributo edad): 3.35 %
Evaluación: no cumplimiento


Se procesa el atributo: **situacion_vivienda**


In [15]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: situacion_vivienda

valores = df_creditos['situacion_vivienda'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['situacion_vivienda']) - df_creditos['situacion_vivienda'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida
    
valores_validos = 'ALQUILER|PROPIA|HIPOTECA|OTROS' # Se define una re de los valores validos según el anexo

df_creditos['situacion_vivienda_ok'] = df_creditos['situacion_vivienda'].astype(str).str.match(valores_validos)

print("Se visualizan las filas con errores de rango:")
display(df_creditos[df_creditos['situacion_vivienda_ok'] == False]) # Para visualizar las tuplas con valores nulos o erróneos

# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['situacion_vivienda_ok'] == False] 
print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
situacion_vivienda
ALQUILER    6125
HIPOTECA    3223
PROPIA       741
OTROS         38
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


Empty DataFrame
Columns: [id_cliente, edad, importe_solicitado, duracion_credito, antiguedad_empleado, situacion_vivienda, ingresos, objetivo_credito, pct_ingreso, tasa_interes, estado_credito, falta_pago, completitud_fila, situacion_vivienda_ok]
Index: []

Cantidad detectada: 0


In [16]:
def calcular_rangos_valores_situacion_vivienda():
    situacion_vivienda_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo situacion_vivienda: {situacion_vivienda_valores_fuera_rango}")
    
    indicador = (situacion_vivienda_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo situacion_vivienda): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_situacion_vivienda()

Cantidad de filas con valores fuera de rango en atributo situacion_vivienda: 0
Porcentaje de filas con errores de rango de valores (atributo situacion_vivienda): 0.0 %
Evaluación: ok


In [17]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: objetivo_credito

valores = df_creditos['objetivo_credito'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['objetivo_credito']) - df_creditos['objetivo_credito'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida
    
valores_validos = 'EDUCACIÓN|SALUD|INVERSIONES|PAGO_DEUDAS|PERSONAL|MEJORAS_HOGAR' 

# Se define una re de los valores validos según el anexo

df_creditos['objetivo_credito_ok'] = df_creditos['objetivo_credito'].astype(str).str.match(valores_validos)

print("Se visualizan las filas con errores de rango:")
display(df_creditos[df_creditos['objetivo_credito_ok'] == False]) # Para visualizar las tuplas con valores nulos o erróneos

# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['objetivo_credito_ok'] == False] 
print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
objetivo_credito
EDUCACIÓN        2328
SALUD            1853
INVERSIONES      1753
PAGO_DEUDAS      1673
PERSONAL         1643
MEJORAS_HOGAR     877
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


Empty DataFrame
Columns: [id_cliente, edad, importe_solicitado, duracion_credito, antiguedad_empleado, situacion_vivienda, ingresos, objetivo_credito, pct_ingreso, tasa_interes, estado_credito, falta_pago, completitud_fila, situacion_vivienda_ok, objetivo_credito_ok]
Index: []

Cantidad detectada: 0


----

Tarjetas

In [18]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: antiguedad_cliente

valores = df_tarjetas['antiguedad_cliente'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_tarjetas['antiguedad_cliente']) - df_tarjetas['antiguedad_cliente'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_tarjetas[df_tarjetas['antiguedad_cliente'] < 0]

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
antiguedad_cliente
36.0    2463
37.0     358
34.0     353
38.0     347
39.0     341
40.0     333
31.0     318
35.0     317
33.0     305
30.0     300
41.0     297
32.0     289
28.0     275
43.0     273
42.0     271
29.0     241
44.0     230
45.0     227
27.0     206
46.0     197
26.0     186
47.0     171
25.0     165
48.0     162
24.0     160
49.0     141
23.0     116
22.0     105
56.0     103
50.0      96
21.0      83
51.0      80
53.0      78
20.0      74
13.0      70
19.0      63
52.0      62
18.0      58
54.0      53
55.0      42
17.0      39
15.0      34
16.0      29
14.0      16
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


Empty DataFrame
Columns: [id_cliente, antiguedad_cliente, estado_civil, estado_cliente, gastos_ult_12m, genero, limite_credito_tc, nivel_educativo, nivel_tarjeta, operaciones_ult_12m, personas_a_cargo, completitud_fila]
Index: []

Cantidad detectada: 0


In [19]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: personas_a_cargo

valores = df_tarjetas['personas_a_cargo'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_tarjetas['personas_a_cargo']) - df_tarjetas['personas_a_cargo'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_tarjetas[df_tarjetas['personas_a_cargo'] < 0]

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
personas_a_cargo
3.0    2732
2.0    2655
1.0    1838
4.0    1574
0.0     904
5.0     424
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


Empty DataFrame
Columns: [id_cliente, antiguedad_cliente, estado_civil, estado_cliente, gastos_ult_12m, genero, limite_credito_tc, nivel_educativo, nivel_tarjeta, operaciones_ult_12m, personas_a_cargo, completitud_fila]
Index: []

Cantidad detectada: 0


- nominales

In [20]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: estado_civil

valores = df_tarjetas['estado_civil'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_tarjetas['estado_civil']) - df_tarjetas['estado_civil'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida
    
valores_validos = 'CASADO|SOLTERO|DESCONOCIDO|DIVORCIADO' # Se define una re de los valores validos según el anexo

df_tarjetas['estado_civil_ok'] = df_tarjetas['estado_civil'].astype(str).str.match(valores_validos)

print("Se visualizan las filas con errores de rango:")
display(df_tarjetas[df_tarjetas['estado_civil_ok'] == False]) # Para visualizar las tuplas con valores nulos o erróneos

# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_tarjetas[df_tarjetas['estado_civil_ok'] == False] 
print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
estado_civil
CASADO         4687
SOLTERO        3943
DESCONOCIDO     749
DIVORCIADO      748
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


Empty DataFrame
Columns: [id_cliente, antiguedad_cliente, estado_civil, estado_cliente, gastos_ult_12m, genero, limite_credito_tc, nivel_educativo, nivel_tarjeta, operaciones_ult_12m, personas_a_cargo, completitud_fila, estado_civil_ok]
Index: []

Cantidad detectada: 0


---

## Dimensión: consistencia

### (4) Claves únicas

Dataset: **datos_creditos**

In [23]:
#Se obtiene el valor de la cantidad de filas actual
cant_antes = df_creditos.shape[0] 

# Se ordena el dataset según el atributo que se desee evaluar (requerido para el paso siguiente)
df_creditos.sort_values("id_cliente", inplace=True)

# Se detectan y eliminan los duplicados en un atributo dejando la última ocurrencia
df_creditos.drop_duplicates(subset ="id_cliente", keep = 'last', inplace = True)

# Se obtiene el valor posterior a la operación
cant_despues = df_creditos.shape[0]

# Se imprimen ambos valores
print('Dataset: creditos')
print(f"Antes del análisis de duplicados: {cant_antes} - Despues del filtrado de duplicados: {cant_despues}")
if cant_antes > cant_despues:
    diferencia = cant_antes - cant_despues
    pct_diferencia = ((cant_antes - cant_despues) / cant_antes) * 100
    print(f"Se detectaron claves duplicadas en {diferencia} fila(s) un {round(pct_diferencia, 2)}%.")
else:
    print("No se detectaron claves duplicadas")

Dataset: creditos
Antes del análisis de duplicados: 10127 - Despues del filtrado de duplicados: 10127
No se detectaron claves duplicadas


Dataset: **datos_tarjetas**

In [22]:
# Se obtiene el valor de la cantidad de filas actual
cant_antes = df_tarjetas.shape[0]  

# Se ordena el dataset según el atributo que se desee evaluar (requerido para el paso siguiente)
df_tarjetas.sort_values("id_cliente", inplace=True)

# Se detectan y eliminan los duplicados en un atributo dejando la última ocurrencia
df_tarjetas.drop_duplicates(subset ="id_cliente", keep = 'last', inplace = True)

# Se obtiene el valor posterior a la operación
cant_despues = df_tarjetas.shape[0]

#Se imprimen ambos valores
print('Dataset: tarjetas')
print(f"Antes del análisis de duplicados: {cant_antes} - Despues del filtrado de duplicados: {cant_despues}")
if cant_antes > cant_despues:
    diferencia = cant_antes - cant_despues
    pct_diferencia = ((cant_antes - cant_despues) / cant_antes) * 100
    print(f"Se detectaron claves duplicadas en {diferencia} fila(s) un {round(pct_diferencia, 2)}%.")
else:
    print("No se detectaron claves duplicadas")

Dataset: tarjetas
Antes del análisis de duplicados: 10127 - Despues del filtrado de duplicados: 10127
No se detectaron claves duplicadas


### (5) Integridad referencial

In [21]:
# Las uniones se hacen de a pares - revisar nombres de atributos

df_integrado = pd.merge(df_creditos, df_tarjetas, on='id_cliente', how='inner')
coincidencias = df_integrado.shape[0]

print(f"Datos de créditos: {cantidad_filas_creditos} - Coincidencias con datos de tarjetas: {coincidencias}")

print("\nSe visualiza el dataset resultante:")
display(df_integrado.head(5))

print(f"Reporte general:\n \
- Filas del dataset creditos (inicial): {cantidad_filas_creditos}\n \
- Filas del dataset tarjetas (inicial): {cantidad_filas_tarjetas}\n \
- Errores detectados en la operación de unión: {abs(coincidencias - cantidad_filas_creditos)} \n \
- Filas del dataset unificado: {df_integrado.shape[0]}")

Datos de créditos: 10127 - Coincidencias con datos de tarjetas: 10127

Se visualiza el dataset resultante:


id_cliente  edad  importe_solicitado  duracion_credito  \
0  713061558.0    22               35000                 3   
1  768805383.0    21                1000                 2   
2  818770008.0    25                5500                 3   
3  713982108.0    23               35000                 2   
4  710821833.0    24               35000                 4   

   antiguedad_empleado situacion_vivienda  ingresos objetivo_credito  \
0                123.0           ALQUILER     59000         PERSONAL   
1                  5.0             PROPIA      9600        EDUCACIÓN   
2                  1.0           HIPOTECA      9600            SALUD   
3                  4.0           ALQUILER     65500            SALUD   
4                  8.0           ALQUILER     54400            SALUD   

   pct_ingreso  tasa_interes  ...  estado_cliente gastos_ult_12m  genero  \
0         0.59         16.02  ...          ACTIVO         1088.0       M   
1         0.10         11.14  ...          ACTIVO         1144.0       M   
2         0.57         12.87  ...          ACTIVO         1291.0       F   
3         0.53         15.23  ...          ACTIVO         1887.0       M   
4         0.55         14.27  ...          ACTIVO         1314.0       M   

   limite_credito_tc         nivel_educativo  nivel_tarjeta  \
0             4010.0  UNIVERSITARIO_COMPLETO           Blue   
1            12691.0     SECUNDARIO_COMPLETO           Blue   
2             8256.0  UNIVERSITARIO_COMPLETO           Blue   
3             3418.0  UNIVERSITARIO_COMPLETO           Blue   
4             9095.0             DESCONOCIDO           Blue   

  operaciones_ult_12m personas_a_cargo  completitud_fila_y estado_civil_ok  
0                24.0              2.0                 0.0            True  
1                42.0              3.0                 0.0            True  
2                33.0              5.0                 0.0            True  
3                20.0              3.0                 0.0            True  
4                26.0              1.0                 0.0            True  

[5 rows x 27 columns]

Reporte general:
 - Filas del dataset creditos (inicial): 10127
 - Filas del dataset tarjetas (inicial): 10127
 - Errores detectados en la operación de unión: 0 
 - Filas del dataset unificado: 10127


In [24]:
def calcular_integridad_referencial():
    cant_problemas = cantidad_filas_creditos - df_integrado.shape[0] # Se calcula sobre el inicio (foco)
    print(f"Casos de problemas de integridad referencial: {cant_problemas}")

    indicador = (cant_problemas / cantidad_filas_creditos)
    print(f"Porcentaje de filas con problemas de integridad referencial: {round(indicador * 100, 2)} %")

    if (indicador > INTEGRIDAD_REF):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')
        
calcular_integridad_referencial()

Casos de problemas de integridad referencial: 0
Porcentaje de filas con problemas de integridad referencial: 0.0 %
Evaluación: ok


---

## Dimensión exactitud (bis)
### (6) Reglas en valores

Regla 1: Para aquellos casos en que los créditos constituyan un porcentaje de los ingresos del cliente mayor al 50% sus ingresos deberán ser mayores a 20.000.

Regla 2: Para aquellos créditos cuya duración sea la mínima permitida el porcentaje de los ingresos del cliente (con respecto al importe solicitado) no podrá exceder el 60% salvo en los casos en los que sea propietario de su vivienda.

In [25]:
#Se puede definir una función para aplicar los cálculos
def regla_pct_ingresos_credito(row):
    pct_ingreso = row.pct_ingreso
    ingresos = row.ingresos
    
    if pct_ingreso > 0.5 and ingresos <= 20000:
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'


# Se aplica la función para todos los elementos del dataset
regla_pct_ingresos = df_integrado.apply(lambda row: regla_pct_ingresos_credito(row), axis=1).rename("regla_pct_ingresos")

# Se unen los resultados al dataset inicial
df_resultado = pd.concat([df_integrado, regla_pct_ingresos], axis=1)  

# Se visualizan los datos
print("Se visualizan las tuplas que no cumplen con la regla:\n")
display(df_resultado[df_resultado.regla_pct_ingresos == 'err'].head())


# Se verifica la cantidad de elementos
aux = df_resultado[df_resultado.regla_pct_ingresos == 'err']
print(f"Cantidad de filas que no cumplen la regla: {aux.shape[0]}")

Se visualizan las tuplas que no cumplen con la regla:



id_cliente  edad  importe_solicitado  duracion_credito  \
2    818770008.0    25                5500                 3   
75   789973308.0    26                6100                 3   
160  718759833.0    23                7750                 4   
460  717392358.0    24               13000                 3   
469  711864783.0    24               10000                 2   

     antiguedad_empleado situacion_vivienda  ingresos objetivo_credito  \
2                    1.0           HIPOTECA      9600            SALUD   
75                   2.0             PROPIA     12000        EDUCACIÓN   
160                  2.0             PROPIA     13920         PERSONAL   
460                  6.0             PROPIA     18000         PERSONAL   
469                  0.0           HIPOTECA     18000      PAGO_DEUDAS   

     pct_ingreso  tasa_interes  ...  gastos_ult_12m genero  limite_credito_tc  \
2           0.57         12.87  ...          1291.0      F             8256.0   
75          0.51          7.51  ...          1341.0      M            24312.0   
160         0.56         17.04  ...          1058.0      F            32643.0   
460         0.72          7.88  ...          1786.0      M            23412.0   
469         0.56         12.18  ...          1199.0      M             2201.0   

              nivel_educativo  nivel_tarjeta  operaciones_ult_12m  \
2      UNIVERSITARIO_COMPLETO           Blue                 33.0   
75   UNIVERSITARIO_INCOMPLETO           Blue                 24.0   
160  UNIVERSITARIO_INCOMPLETO         Silver                 24.0   
460    UNIVERSITARIO_COMPLETO           Blue                 50.0   
469  UNIVERSITARIO_INCOMPLETO           Blue                 32.0   

    personas_a_cargo completitud_fila_y  estado_civil_ok regla_pct_ingresos  
2                5.0                0.0             True                err  
75               1.0                0.0             True                err  
160              2.0                0.0             True                err  
460              1.0                0.0             True                err  
469              1.0                0.0             True                err  

[5 rows x 28 columns]

Cantidad de filas que no cumplen la regla: 15


Regla 2: Para aquellos créditos cuya duración sea la mínima permitida el porcentaje de los ingresos del cliente (con respecto al importe solicitado) no podrá exceder el 60% salvo en los casos en los que sea propietario de su vivienda.

In [26]:
#Se puede definir una función para aplicar los cálculos
def regla_pct_ingresos_credito(row):
    pct_ingreso = row.pct_ingreso
    duracion_credito = row.duracion_credito
    situacion_vivienda = row.situacion_vivienda

    
    if duracion_credito == 2 and pct_ingreso > 0.6 and situacion_vivienda == "ALQUILER":
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'


# Se aplica la función para todos los elementos del dataset
regla_pct_ingresos = df_integrado.apply(lambda row: regla_pct_ingresos_credito(row), axis=1).rename("regla_pct_ingresos")

# Se unen los resultados al dataset inicial
df_resultado = pd.concat([df_integrado, regla_pct_ingresos], axis=1)  

# Se visualizan los datos
print("Se visualizan las tuplas que no cumplen con la regla:\n")
display(df_resultado[df_resultado.regla_pct_ingresos == 'err'].head())


# Se verifica la cantidad de elementos
aux = df_resultado[df_resultado.regla_pct_ingresos == 'err']
print(f"Cantidad de filas que no cumplen la regla: {aux.shape[0]}")

Se visualizan las tuplas que no cumplen con la regla:



id_cliente  edad  importe_solicitado  duracion_credito  \
36    713962233.0    22               30000                 2   
577   716027208.0    26               20050                 2   
587   709186983.0    23               20000                 2   
6758  713115483.0    22                7000                 2   

      antiguedad_empleado situacion_vivienda  ingresos objetivo_credito  \
36                    1.0           ALQUILER     48000        EDUCACIÓN   
577                  10.0           ALQUILER     26000        EDUCACIÓN   
587                   1.0           ALQUILER     32900        EDUCACIÓN   
6758                  0.0           ALQUILER     10000    MEJORAS_HOGAR   

      pct_ingreso  tasa_interes  ...  gastos_ult_12m genero  \
36           0.63         18.39  ...          1877.0      F   
577          0.77          9.76  ...          1110.0      M   
587          0.61         16.00  ...          1536.0      M   
6758         0.70         12.21  ...          2275.0      F   

      limite_credito_tc           nivel_educativo  nivel_tarjeta  \
36               3035.0    UNIVERSITARIO_COMPLETO           Blue   
577             19665.0         POSGRADO_COMPLETO           Blue   
587              8714.0  UNIVERSITARIO_INCOMPLETO           Blue   
6758             1684.0  UNIVERSITARIO_INCOMPLETO           Blue   

      operaciones_ult_12m personas_a_cargo completitud_fila_y  \
36                   37.0              3.0                0.0   
577                  58.0              4.0                0.0   
587                  38.0              4.0                0.0   
6758                 45.0              3.0                0.0   

      estado_civil_ok regla_pct_ingresos  
36               True                err  
577              True                err  
587              True                err  
6758             True                err  

[4 rows x 28 columns]

Cantidad de filas que no cumplen la regla: 4


In [27]:
df_integrado.to_csv("../../data/processed/datos_integrados.mc.csv", sep=";", index=False)